## Translation Experiments

#### Importing Relevant Models

In [1]:
import os
import sys
sys.path.append('../')

from dotenv import load_dotenv
load_dotenv("../envs/melange.env")

True

In [2]:
from mega.models.completion_models import gpt3x_completion
from langchain.prompts import load_prompt
from langchain.prompts.example_selector import (
    MaxMarginalRelevanceExampleSelector, 
    SemanticSimilarityExampleSelector
)
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import FewShotPromptTemplate

c:\Users\t-iwatts\AppData\Local\miniconda3\envs\mega\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### In-22 Gen

#### Dataset Loading

In [3]:
files = [file for file in os.listdir('../translation_dataset/IN22_testset/gen') if file.startswith('test')]
files

['test.asm_Beng',
 'test.ben_Beng',
 'test.brx_Deva',
 'test.doi_Deva',
 'test.eng_Latn',
 'test.gom_Deva',
 'test.guj_Gujr',
 'test.hin_Deva',
 'test.kan_Knda',
 'test.kas_Arab',
 'test.mai_Deva',
 'test.mal_Mlym',
 'test.mar_Deva',
 'test.mni_Mtei',
 'test.npi_Deva',
 'test.ory_Orya',
 'test.pan_Guru',
 'test.san_Deva',
 'test.sat_Olck',
 'test.snd_Deva',
 'test.tam_Taml',
 'test.tel_Telu',
 'test.urd_Arab']

In [4]:
dataset = dict()

for file in files:
    language = file[5:]
    dataset[language] = dict()

    with open(os.path.join('../translation_dataset/IN22_testset/gen', file), mode='r', encoding='utf-8') as f:
        data = [line.strip() for line in f.readlines()]

    for i, val in enumerate(data):
        dataset[language][i] = val

In [5]:
dataset.keys()

dict_keys(['asm_Beng', 'ben_Beng', 'brx_Deva', 'doi_Deva', 'eng_Latn', 'gom_Deva', 'guj_Gujr', 'hin_Deva', 'kan_Knda', 'kas_Arab', 'mai_Deva', 'mal_Mlym', 'mar_Deva', 'mni_Mtei', 'npi_Deva', 'ory_Orya', 'pan_Guru', 'san_Deva', 'sat_Olck', 'snd_Deva', 'tam_Taml', 'tel_Telu', 'urd_Arab'])

In [6]:
dataset['asm_Beng']

{0: 'এটা বাহ্যিক ৰূপ হৈছে সশস্ত্র বাহিনীৰ জোৱানগৰাকীৰ সৈতে জড়িত কেইবাটাও বৈশিষ্ট, যেনে তেওঁৰ জোতা, কাপোৰ, টাই, অলংকাৰ, কেশ-বিন্যাস, মেক-আপ, ঘড়ী, প্রসাধনী, সুগন্ধি, ইত্যাদি।',
 1: 'মহাৰাষ্ট্রৰ ঔৰঙ্গাবাদ জিলাত অৱস্থিত অজন্তাত খ্রীষ্টপূর্ব প্রথম শতিকাৰ পৰা খ্রীষ্টাব্দ পঞ্চম শতিকাৰ ভাস্কর্য্য আৰু চিত্রৰে শোভিত ঊনত্রিশটা চৈত্য আৰু বিহাৰ গুহা আছে।',
 2: 'দেহৰ ৰং বাহিৰত থকা ৰেখাডালৰ লগত মিলি যায়, যিয়ে ভলিউমৰ ইফেক্টটো সৃষ্টি কৰে।',
 3: 'অশোকে ভাস্কর্য্য আৰু বিশাল স্মাৰকসমূহৰ বাবে শিলৰ ব্যাপক প্রয়োগ আৰম্ভ কৰাৰ বিপৰীতে পূর্বৰ পৰম্পৰা আছিল কাঠ আৰু মাটিৰে কাম কৰা।',
 4: 'মচলাৰ সৈতে মিহলোৱা, বেচনৰ ঘোলৰ তৰপ দিয়া আৰু অত্যুত্তম অৱস্থালৈ তেলত ডুবাই ভজা আলুৱে মহাৰাষ্ট্ৰৰ এইবিধ সুস্বাদু তথা বিখ্যাত খাদ্য তৈয়াৰ কৰে।',
 5: 'চেট্টিনাড ব্যঞ্জন তামিলনাডু ৰাজ্যৰ চেট্টিনাড অঞ্চলৰ নাট্টুকোটাই চেট্টিৰাছ নামৰ এটা সম্প্ৰদায় বা নাগাৰাঠাৰাছ, যেনেকৈ তেওঁলোকে নিজকে কয়, সেই সম্প্রদায়ৰ ব্যঞ্জন।',
 6: 'ডালচেনি ভাৰতৰ থলুৱা আৰু সেইবাবেই এবিধ সোৱাদবর্ধক বস্তুৰ পৰা আদি কৰি এবিধ ঔষধি দ্রব্যলৈকে ইয়াক ব্যৱহাৰ কৰা হয়।'

#### Few Shot Dataset

In [7]:
few_shot_dataset = dict()

In [8]:
directory_path = '../translation_dataset/flores-22_dev/all'

for folder in os.listdir(directory_path):
    few_shot_dataset[folder[9:]] = list()
    for file in os.listdir(os.path.join(directory_path, folder)):
        if file[-4:] != '.csv':
            if file == 'dev.eng_Latn':
                with open(os.path.join(directory_path, folder, file), mode='r', encoding='utf-8') as file_eng:
                    data_eng = [val.strip() for val in file_eng.readlines()]
            else:
                with open(os.path.join(directory_path, folder, file), mode='r', encoding='utf-8') as file_source:
                    data_source = [val.strip() for val in file_source.readlines()]

    for source, english in zip(data_source, data_eng):
        few_shot_dataset[folder[9:]].append({
            'sentence' : source,
            'response' : english
        })            

In [9]:
few_shot_dataset['asm_Beng']

[{'sentence': "সোমবাৰে, ষ্টেনফ'ৰ্ড ইউনিভাৰচিটি স্কুল অৱ মেডিচিনৰ বিজ্ঞানীসকলে  ঘোষণা কৰিছিল যে তেওঁলোকে এটা নতুন ৰোগ চিনাক্তকৰণ সঁজুলি আৱিষ্কাৰ কৰিছিল যিয়ে ধৰণ অনুসৰি কোষসমূহ ক্ৰম কৰে: এটা ক্ষুদ্ৰ মুদ্ৰণযোগ্য চিপ যিটো প্ৰত্যেকটোতে সম্ভৱপৰ প্ৰায় 1 U.S. চেণ্ট খৰচ হোৱাকৈ ষ্টেণ্ডাৰ্ড ইনজেক্ট প্ৰিণ্টাৰ ব্যৱহাৰ কৰি প্ৰস্তুত কৰা হৈছে।",
  'response': 'On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.'},
 {'sentence': 'শীৰ্ষ গৱেষকে কয় যে এইটোৱে কেন্সাৰ, যক্ষ্মা, এইচ.আই.ভি আৰু মেলেৰিয়াৰ প্ৰাথমিক পৰ্য্যায়ৰ চিনাক্তকৰণ নিম্ন আয়ৰ দেশবিলাকৰ ৰোগীসকললৈ আনিব পাৰে , য’ত স্তন কেন্সাৰৰ দৰে বেমাৰবোৰৰ সুস্থ হোৱাৰ হাৰ ধনী দেশৰ তুলনাত অৰ্ধেক হ’ব পাৰে।',
  'response': 'Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries,

### Prompting Templates

In [10]:
class PromptTemplate:

    def __init__(self, source, target, sentence) -> None:
        self.source = source
        self.target = target
        self.sentence = sentence

    def get_example_selector(self, criteria, few_shots, top_k):
        
        embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/LaBSE')

        if criteria == 'semantic':
            return SemanticSimilarityExampleSelector.from_examples(
                examples=few_shots, 
                embeddings=embeddings, 
                vectorstore_cls=FAISS, 
                k = top_k
            )
        
        elif criteria == 'mmr':
            return MaxMarginalRelevanceExampleSelector.from_examples(
                examples=few_shots, 
                embeddings=embeddings, 
                vectorstore_cls=Chroma, 
                k = top_k
            )

    def get_query_prompt(self):
        query_prompt = load_prompt('../mega/prompting/translation_template/query.yaml')
        query_prompt = query_prompt.format(source=self.source, target=self.target, sentence=self.sentence)
        return query_prompt
    
    def get_few_shot_prompt(self, few_shots, criteria, top_k=8):
        example_prompt = load_prompt("../mega/prompting/translation_template/few_shot.yaml")

        prefix = """INSTRUCTION: Translate the following Sentence from {source} to {target} in the example format given below.
        """
        suffix = "Sentence : {sentence}\nResponse :"
        
        few_shot_prompt = FewShotPromptTemplate(
            example_selector=self.get_example_selector(criteria, few_shots, top_k),
            examples=None,
            example_prompt=example_prompt,
            prefix=prefix,
            suffix=suffix,
            input_variables=['source', 'target', 'sentence'],
            example_separator="\n"
        )

        return few_shot_prompt.format(source=self.source, target=self.target, sentence=self.sentence)

In [11]:
source = 'Hindi'
target = 'English'
sentence = dataset['hin_Deva'][0]
sentence

'सेवा संबंधी लोगों के लिए भेष कई गुणों का संयोजन है, जैसे कि उनके जूते, कपड़े, टाई, आभूषण, केश शैली, मेक-अप, घड़ी, कॉस्मेटिक, इत्र, आदि।'

In [12]:
few_shots = few_shot_dataset['hin_Deva']
len(few_shots)

997

In [13]:
prompts = PromptTemplate(source, target, sentence)

In [14]:
prompts.get_query_prompt()

'Instruction : Translate the following Sentence from Hindi to English.\nSentence: सेवा संबंधी लोगों के लिए भेष कई गुणों का संयोजन है, जैसे कि उनके जूते, कपड़े, टाई, आभूषण, केश शैली, मेक-अप, घड़ी, कॉस्मेटिक, इत्र, आदि।\nResponse:'

In [15]:
prompt = prompts.get_few_shot_prompt(few_shots[:8], 'mmr', 2)
prompt

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
response = gpt3x_completion(prompt, 'gpt-35-turbo')
response

'Costume is a composite of many elements for people in the service industry, including footwear, clothing, ties'